In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from numpy.linalg import norm
from random import normalvariate
from math import sqrt

In [2]:
DF = pd.read_excel('dataset.xlsx')
DF.drop(DF.columns[[0, 1, 2, 3, 4, 5]], axis=1, inplace=True) # Delete unnecessary columns
DF.drop_duplicates(keep=False, inplace=True) # Remove duplicate rows
len(DF)

FileNotFoundError: [Errno 2] No such file or directory: 'dataset.xlsx'

In [ ]:
DF.fillna(0.5, inplace=True)
len(DF)

128

# SVD

In [ ]:
A = DF
A.head()

,Data Structures and Algorithms,Computer Architecture\n,Discrete Mathematics\n,Economics,Programming-2,Machine Learning\n,Mathematics For Machine Learning\n,Visual Recognition\n,Natural Language Processing\n,Reinforcement Learning,...,Digital CMOS VLSI Design,System design with FPGA,ASIC design,VLSI Architecture Design\n,High level synthesis and optimization of Digital Circuits,Digital Sociology,Privacy in the Digital Age,Technology Ethics and AI,Techno-economics of networks,The Web and the Mind
0,0.30,0.85,0.85,0.90,0.70,0.70,0.80,0.50,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.6,0.5,0.5,0.5
1,0.70,0.80,0.20,0.80,0.80,0.50,0.10,0.20,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5,0.5
2,0.70,0.00,1.00,1.00,1.00,1.00,1.00,0.80,0.9,1.0,...,0.5,0.5,0.5,0.5,0.5,0.5,0.4,0.5,0.5,0.5
3,0.65,0.70,0.20,0.75,0.75,0.75,0.75,0.75,0.5,0.5,...,0.5,0.5,0.5,0.5,0.5,0.8,0.5,0.5,0.8,0.5
4,0.70,0.60,0.20,0.00,0.80,0.50,0.60,0.40,0.4,0.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
A = A.to_numpy()
A

array([[0.3 , 0.85, 0.85, ..., 0.5 , 0.5 , 0.5 ],
       [0.7 , 0.8 , 0.2 , ..., 0.5 , 0.5 , 0.5 ],
       [0.7 , 0.  , 1.  , ..., 0.5 , 0.5 , 0.5 ],
       ...,
       [1.  , 0.3 , 1.  , ..., 0.4 , 0.6 , 0.3 ],
       [0.9 , 0.7 , 0.8 , ..., 0.5 , 0.5 , 0.5 ],
       [0.2 , 0.8 , 0.7 , ..., 0.5 , 0.5 , 0.5 ]])

In [ ]:
def QR_Decomposition(A):
    n, m = A.shape # get the shape of A

    Q = np.empty((n, n)) # initialize matrix Q
    u = np.empty((n, n)) # initialize matrix u

    u[:, 0] = A[:, 0]
    Q[:, 0] = u[:, 0] / np.linalg.norm(u[:, 0])

    for i in range(1, n):

        u[:, i] = A[:, i]
        for j in range(i):
            u[:, i] -= (A[:, i] @ Q[:, j]) * Q[:, j] # get each u vector

        Q[:, i] = u[:, i] / np.linalg.norm(u[:, i]) # compute each e vetor

    R = np.zeros((n, m))
    for i in range(n):
        for j in range(i, m):
            R[i, j] = A[:, j] @ Q[:, i]

    return Q, R

In [ ]:

def svd_simultaneous_power_iteration(A, k, epsilon=0.00001):
    n_orig, m_orig = A.shape
    if k is None:
        k=min(n_orig,m_orig)
    A_orig=A.copy()
    if n_orig > m_orig:
        A = A.T @ A
        n, m = A.shape
    elif n_orig < m_orig:
        A = A @ A.T
        n, m = A.shape
    else:
        n,m=n_orig, m_orig
        
    Q = np.random.rand(n, k)
    # Q, _ = np.linalg.qr(Q)
    Q, _ = QR_Decomposition(Q)
    Q_prev = Q
    #this part does the block power iteration
    for i in range(1000):
        Z = A @ Q
        # Q, R = np.linalg.qr(Z)
        Q, R = QR_Decomposition(Z)
        err = ((Q - Q_prev) ** 2).sum()
        Q_prev = Q
        if err < epsilon:
            break
            
    singular_values=np.sqrt(np.diag(R))
    # print(singular_values)
    # singular_values = singular_values[:-5]
    # print(singular_values)
    #deal with different shape input matrices
    if n_orig < m_orig: 
        left_vecs=Q.T
        # left_vecs = left_vecs[:, :-5]
        print(left_vecs.shape)
        #use property Values @ V = U.T@A => V=inv(Values)@U.T@A
        right_vecs=np.linalg.inv(np.diag(singular_values))@left_vecs.T@A_orig
    elif n_orig==m_orig:
        left_vecs=Q.T
        print(left_vecs.shape)
        right_vecs=left_vecs
        singular_values=np.square(singular_values)
    else:
        right_vecs=Q.T
        # right_vecs = right_vecs[:-5, :]
        print(right_vecs.shape)
        #use property Values @ V = U.T@A => U=A@V@inv(Values)
        left_vecs=A_orig@ right_vecs.T @np.linalg.inv(np.diag(singular_values))

    print(left_vecs.shape)
    print(singular_values.shape)
    print(right_vecs.shape)
    return left_vecs, singular_values, right_vecs

In [ ]:
tryy = np.array([[0.6, 0.4, 0.9], 
                 [0.3, 0.1, 0.7], 
                 [0.1, 0.5, 0.8], 
                 [0.55, 0.45, 0.5], 
                 [0.7, 0.7 , 0.7]])

In [ ]:
lv, sv, rv = svd_simultaneous_power_iteration(tryy, None)

(3, 3)
(5, 3)
(3,)
(3, 3)


In [ ]:
sv

array([2.16618585, 0.48567319, 0.34170149])

In [ ]:
Sigma = np.zeros((lv.shape[1], rv.shape[0]))
Sigma[:A.shape[1], :A.shape[1]] = np.diag(sv)
Sigma

array([[2.16618585, 0.        , 0.        ],
       [0.        , 0.48567319, 0.        ],
       [0.        , 0.        , 0.34170149]])

In [ ]:
test = lv @ Sigma @ rv
test

array([[0.6 , 0.4 , 0.9 ],
       [0.3 , 0.1 , 0.7 ],
       [0.1 , 0.5 , 0.8 ],
       [0.55, 0.45, 0.5 ],
       [0.7 , 0.7 , 0.7 ]])

In [ ]:
tryy

array([[0.6 , 0.4 , 0.9 ],
       [0.3 , 0.1 , 0.7 ],
       [0.1 , 0.5 , 0.8 ],
       [0.55, 0.45, 0.5 ],
       [0.7 , 0.7 , 0.7 ]])

In [ ]:
lv

array([[ 0.52805028,  0.09793614,  0.40691597],
       [ 0.32815136,  0.453961  ,  0.55604906],
       [ 0.40304807,  0.59463447, -0.6821535 ],
       [ 0.3906995 , -0.40476709,  0.0268203 ],
       [ 0.54624696, -0.51662914, -0.24325695]])

In [ ]:
u, s, v = np.linalg.svd(tryy, full_matrices=False)
u, s, v

(array([[-0.52805028, -0.09851876, -0.4066329 ],
        [-0.32815136, -0.45475546, -0.55473543],
        [-0.40304807, -0.59365377,  0.68387579],
        [-0.3906995 ,  0.40472652, -0.02799216],
        [-0.54624696,  0.51697502,  0.24176154]]),
 array([2.16618585, 0.48567421, 0.34170077]),
 array([[-0.48603308, -0.46337064, -0.74098549],
        [ 0.67860064,  0.33417363, -0.65408651],
        [-0.55070229,  0.82074091, -0.15202418]]))

In [ ]:
u @ u.T

array([[ 0.45389336,  0.44365603, -0.00677072,  0.17781836,  0.13920593],
       [ 0.44365603,  0.62221724,  0.02285794, -0.04031478, -0.18995922],
       [-0.00677072,  0.02285794,  0.98255863, -0.10193991,  0.07859448],
       [ 0.17781836, -0.04031478, -0.10193991,  0.31723321,  0.41588449],
       [ 0.13920593, -0.18995922,  0.07859448,  0.41588449,  0.62409756]])

In [ ]:
np.allclose(A, test)

ValueError: operands could not be broadcast together with shapes (128,25) (5,3) 